## Librerías a Usar


In [ ]:
!pip install -q \
  torch==2.2.1 \
  pyodbc \
  langdetect \
  datasets \
  transformers \
  accelerate \
  einops \
  langchain \
  xformers bitsandbytes \
  faiss-gpu \
  sentence_transformers \
  dill==0.3.7 \
  multiprocess==0.70.15 \
  googletrans==3.1.0a0 \
  torchaudio \
  torchtext \
  torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 34.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134

In [ ]:
!pip show accelerate

Name: accelerate
Version: 0.30.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.43.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [ ]:
!pip show transformers

Name: transformers
Version: 4.40.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: sentence-transformers


In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.4 MB/s eta 0:00:00


In [ ]:
import os
import warnings
from datetime import datetime

import re
import pyodbc
import spacy

import numpy as np
import pandas as pd

import nltk
import openpyxl

import wordcloud
from wordcloud import WordCloud

import matplotlib.pyplot as plt

from langdetect import detect

import unicodedata

from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

from string import digits

import torch
from torch import cuda, bfloat16

import transformers
from transformers import pipeline, StoppingCriteria, StoppingCriteriaList

from langchain import HuggingFacePipeline, PromptTemplate, LLMChain

from datasets import Dataset

from tqdm import tqdm
from google.colab import drive

from googletrans import Translator, constants
from pprint import pprint

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('spanish'))
def preprocess_text(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
    return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## MODELO Mistral

### Parámetros y funciones

In [ ]:
from dotenv import load_dotenv
import os

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API desde las variables de entorno
HF_AUTH = os.getenv("HF_AUTH")

In [ ]:
MODEL_ID = 'mistralai/Mistral-7B-Instruct-v0.2'
STOP_LIST = ['\n```\n']
DEVICE = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(DEVICE)

cuda:0


In [ ]:
# Definir tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    MODEL_ID,
    token=HF_AUTH
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
# Function to define stop tokens
def define_stop_tokens(stop_list: list[str],
                       tokenizer: transformers.AutoTokenizer,
                       device: str) -> list[torch.LongTensor]:
    stop_token_ids = [tokenizer(stop_word)['input_ids'] for stop_word in stop_list]
    stop_token_tensors = [torch.LongTensor(token_ids).to(device) for token_ids in stop_token_ids]
    return stop_token_tensors

# Define custom stopping criteria object
class StopOnTokens(StoppingCriteria):

  def __init__(self, stop_token_ids):
    self.stop_token_ids = stop_token_ids

  def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
      for stop_ids in self.stop_token_ids:
          if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
              return True
      return False

stop_token_ids = define_stop_tokens(STOP_LIST, tokenizer, DEVICE)
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids)])

In [ ]:
def cargar_modelo() -> transformers.AutoModelForCausalLM:
  # Set quantization configuration to load large model with less GPU memory
  # This requires the `bitsandbytes` library
  bnb_config = transformers.BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type='nf4',
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=bfloat16
  )

  # Begin initializing HF items, you need an access token
  modelo_config = transformers.AutoConfig.from_pretrained(
      MODEL_ID,
      token=HF_AUTH
  )

  # Cargar transformer
  modelo = transformers.AutoModelForCausalLM.from_pretrained(
      MODEL_ID,
      trust_remote_code=True,
      config=modelo_config,
      quantization_config=bnb_config,
      device_map='auto',
      token=HF_AUTH
  )

  # Enable evaluation mode to allow model inference
  modelo.eval()

  return modelo

modelo = cargar_modelo()

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
def crear_pipeline(
    modelo: transformers.AutoModelForCausalLM,
    tokenizer: transformers.AutoTokenizer,
    stopping_criteria: StoppingCriteriaList) -> HuggingFacePipeline:

  # Definir pipeline
  pipe = transformers.pipeline(
    model=modelo,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    stopping_criteria=stopping_criteria, # without this model rambles during chat
    temperature=0.05,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=300,  # max number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
    do_sample=True, # Añadido para el mistral
    eos_token_id=tokenizer.eos_token_id, # Añadido para el mistral
    pad_token_id=tokenizer.eos_token_id # Añadido para el mistral
  )

  # Definir LLM
  llm = HuggingFacePipeline(pipeline = pipe)

  return llm

llm = crear_pipeline(modelo=modelo, tokenizer=tokenizer, stopping_criteria=stopping_criteria)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


## RAG

### Basic RAG

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
txt_path = '/content/drive/MyDrive/Data/codigos_RAG.txt'

In [ ]:
# Abre el archivo en modo lectura ('r')
with open(txt_path, 'r') as file:
    # Lee cada línea del archivo y lo guarda en una lista
    lineas = file.readlines()

# Ahora puedes imprimir o manipular cada línea según tus necesidades
for linea in lineas:
    print(linea)


R53.1    Weakness

M62.81   Muscle weakness (generalized)

R2.50    Abnormal head movements

R25.2    Cramp and spasm

R27.0    Ataxia, unspecified

R27.8    Other lack of coordination

R27.9    Unspecified lack of coordinatioN

R47.1    Aphasia

R47.81   Slurred speech

R47.82   Fluency disorder in conditions classified elsewhere

R47.89   Other speech disturbances

R47.9    Unspecified speech disturbances

R131.0   Dysphagia, unspecified

R05.1    Acute cough

R06.0    Dyspnea

R06.00   Dyspnea, unspecified

R06.02   Shortness of breath

R07      Pain in throat and thorax

R07.0    Pain in throat

R07.9    Chest pain, unspecified

R25.3    Fasciculation

F32      Depressive episode

F32.A    Depression, unspecified

F41.0    Panic disorder [episodic paroxysmal anxiety]

F41.9    Anxiety disorder, unspecified

R41.840  Attention and concentration deficit

R41.3    Other amnesia

R53.82   Chronic fatigue, unspecified

R53.83   Other fatigue

R29.90   Unspecified symptoms and signs invo

#### Chunks

In [ ]:
chunk_size = 1
chunks = [lineas[i : i + chunk_size] for i in range(0, len(lineas), chunk_size)]

In [ ]:
len(chunks)

87

In [ ]:
chunks

[['\ufeffR53.1    Weakness\n'],
 ['M62.81   Muscle weakness (generalized)\n'],
 ['R2.50    Abnormal head movements\n'],
 ['R25.2    Cramp and spasm\n'],
 ['R27.0    Ataxia, unspecified\n'],
 ['R27.8    Other lack of coordination\n'],
 ['R27.9    Unspecified lack of coordinatioN\n'],
 ['R47.1    Aphasia\n'],
 ['R47.81   Slurred speech\n'],
 ['R47.82   Fluency disorder in conditions classified elsewhere\n'],
 ['R47.89   Other speech disturbances\n'],
 ['R47.9    Unspecified speech disturbances\n'],
 ['R131.0   Dysphagia, unspecified\n'],
 ['R05.1    Acute cough\n'],
 ['R06.0    Dyspnea\n'],
 ['R06.00   Dyspnea, unspecified\n'],
 ['R06.02   Shortness of breath\n'],
 ['R07      Pain in throat and thorax\n'],
 ['R07.0    Pain in throat\n'],
 ['R07.9    Chest pain, unspecified\n'],
 ['R25.3    Fasciculation\n'],
 ['F32      Depressive episode\n'],
 ['F32.A    Depression, unspecified\n'],
 ['F41.0    Panic disorder [episodic paroxysmal anxiety]\n'],
 ['F41.9    Anxiety disorder, unspecified\n

In [ ]:
flattened_chunks = [chunk[0] for chunk in chunks]
flattened_chunks

['\ufeffR53.1    Weakness\n',
 'M62.81   Muscle weakness (generalized)\n',
 'R2.50    Abnormal head movements\n',
 'R25.2    Cramp and spasm\n',
 'R27.0    Ataxia, unspecified\n',
 'R27.8    Other lack of coordination\n',
 'R27.9    Unspecified lack of coordinatioN\n',
 'R47.1    Aphasia\n',
 'R47.81   Slurred speech\n',
 'R47.82   Fluency disorder in conditions classified elsewhere\n',
 'R47.89   Other speech disturbances\n',
 'R47.9    Unspecified speech disturbances\n',
 'R131.0   Dysphagia, unspecified\n',
 'R05.1    Acute cough\n',
 'R06.0    Dyspnea\n',
 'R06.00   Dyspnea, unspecified\n',
 'R06.02   Shortness of breath\n',
 'R07      Pain in throat and thorax\n',
 'R07.0    Pain in throat\n',
 'R07.9    Chest pain, unspecified\n',
 'R25.3    Fasciculation\n',
 'F32      Depressive episode\n',
 'F32.A    Depression, unspecified\n',
 'F41.0    Panic disorder [episodic paroxysmal anxiety]\n',
 'F41.9    Anxiety disorder, unspecified\n',
 'R41.840  Attention and concentration deficit

#### Embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('michiyasunaga/BioLinkBERT-base')
model = AutoModel.from_pretrained('michiyasunaga/BioLinkBERT-base')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/447k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(flattened_chunks, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
len(last_hidden_states)

87

In [ ]:
text_embeddings = []
for last_hidden_state in last_hidden_states:
  text_embeddings.append(last_hidden_state.detach().numpy())


In [ ]:
len(text_embeddings)

87

In [ ]:
len(text_embeddings[0])

19

In [ ]:
last_hidden_states

tensor([[[ 1.6254e-01,  1.0784e-01,  3.6217e-01,  ..., -5.8507e-01,
           1.9825e-02, -7.9442e-02],
         [ 3.1387e-01,  2.2871e-01, -8.1193e-02,  ...,  1.9693e-01,
           8.0268e-01, -2.5984e-01],
         [ 4.7435e-02,  5.2670e-01,  1.0955e-01,  ...,  5.7361e-01,
          -7.2667e-01,  3.4105e-03],
         ...,
         [-3.5715e-01,  7.1252e-01, -2.4119e-01,  ...,  1.6578e-01,
           3.9572e-01, -2.6114e-02],
         [-3.6083e-01,  7.0814e-01, -2.4218e-01,  ...,  1.6218e-01,
           3.9735e-01, -3.1996e-02],
         [-3.5916e-01,  7.1375e-01, -2.4293e-01,  ...,  1.5619e-01,
           4.0195e-01, -2.2964e-02]],

        [[ 2.7553e-01,  1.4439e-01,  2.6749e-01,  ..., -5.6307e-01,
          -3.1763e-01,  1.3722e-01],
         [ 2.2248e-01,  3.0401e-01, -1.1953e-01,  ..., -3.6124e-01,
           1.6035e+00,  6.0988e-02],
         [ 1.3149e-01,  6.0324e-01, -1.0349e-01,  ...,  1.3762e-01,
          -1.2744e-01, -3.4047e-01],
         ...,
         [-1.1633e-01,  6

In [ ]:
text_embeddings

[array([[ 0.16253522,  0.10783777,  0.36216575, ..., -0.58506775,
          0.01982479, -0.07944173],
        [ 0.3138677 ,  0.22870642, -0.08119291, ...,  0.1969258 ,
          0.8026772 , -0.25984347],
        [ 0.0474354 ,  0.52670157,  0.10954889, ...,  0.57361364,
         -0.7266687 ,  0.00341053],
        ...,
        [-0.35714552,  0.7125215 , -0.24119183, ...,  0.16578116,
          0.39571813, -0.02611391],
        [-0.36083248,  0.7081413 , -0.2421833 , ...,  0.16218452,
          0.39735046, -0.03199647],
        [-0.3591571 ,  0.7137481 , -0.2429342 , ...,  0.15619199,
          0.40195295, -0.02296378]], dtype=float32),
 array([[ 0.27553028,  0.14438964,  0.26749128, ..., -0.5630721 ,
         -0.31763074,  0.13721907],
        [ 0.22247769,  0.30401322, -0.11953458, ..., -0.36124444,
          1.6034657 ,  0.06098811],
        [ 0.13148943,  0.6032424 , -0.10348993, ...,  0.13761698,
         -0.12744354, -0.34046578],
        ...,
        [-0.11632828,  0.6075089 , -0.4

#### VDB Faiss

In [ ]:
# import faiss

# d = text_embeddings[0].shape[0]
# index = faiss.IndexFlatL2(d)

# for i, vec in enumerate(text_embeddings):
#     assert len(vec) == d, f"El vector {i} no tiene la misma dimensión que el primer vector."
#     print(len(vec), d)
#     index.add(vec)


### RAG LangChain

En esta seccion se generan los chunks y se guardan en la base de datos vectorial. Se volverá a usar este contenido al llamar al retriever de la base de datos vectorial (retriever from the vector store. The retriever is used to search for similar chunks based on queries).

#### DocumentLoader: docs

In [ ]:
!pip install langchain

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csv_path = '/content/drive/MyDrive/Data/codigos_cie_format.csv'

In [ ]:
import re
import pandas as pd


In [ ]:
# pattern = re.compile(r'\s{2,}|\t')
# with open(txt_path, 'r') as archivo:
#     lineas = archivo.readlines()
# data = [pattern.split(line.strip()) for line in lineas]
# df = pd.DataFrame(data, columns=['Code', 'Description'])
df = pd.read_csv(csv_path)
df['Code'] = df['Code'].str.strip()
df['Description'] = df['Description'].str.strip()
df = df.dropna(subset=['Code', 'Description'])
df = df.reset_index(drop=True)
df.head()

,Code,Description
0,A00.0,"cholera due to vibrio cholerae 01, biovar chol..."
1,A00.1,"cholera due to vibrio cholerae 01, biovar eltor"
2,A00.9,"cholera, unspecified"
3,A01.00,"typhoid fever, unspecified"
4,A01.01,typhoid meningitis


In [ ]:
df.shape

(22903, 2)

In [ ]:
code = df["Code"].tolist()
description = df["Description"].tolist()

new_unique_id = [f"{x}_{y}" for x, y in zip(code, description)]

In [ ]:
from langchain.docstore.document import Document
docs = [
        Document(page_content=x, metadata={"ids": id}) for x, id in zip(description, new_unique_id)
    ]

In [ ]:
docs

[Document(page_content='cholera due to vibrio cholerae 01, biovar cholerae', metadata={'ids': 'A00.0_cholera due to vibrio cholerae 01, biovar cholerae'}),
 Document(page_content='cholera due to vibrio cholerae 01, biovar eltor', metadata={'ids': 'A00.1_cholera due to vibrio cholerae 01, biovar eltor'}),
 Document(page_content='cholera, unspecified', metadata={'ids': 'A00.9_cholera, unspecified'}),
 Document(page_content='typhoid fever, unspecified', metadata={'ids': 'A01.00_typhoid fever, unspecified'}),
 Document(page_content='typhoid meningitis', metadata={'ids': 'A01.01_typhoid meningitis'}),
 Document(page_content='typhoid fever with heart involvement', metadata={'ids': 'A01.02_typhoid fever with heart involvement'}),
 Document(page_content='typhoid pneumonia', metadata={'ids': 'A01.03_typhoid pneumonia'}),
 Document(page_content='typhoid arthritis', metadata={'ids': 'A01.04_typhoid arthritis'}),
 Document(page_content='typhoid osteomyelitis', metadata={'ids': 'A01.05_typhoid oste

In [ ]:
len(docs)

22903

#### Text Splitter y VDB Chroma

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 

In [ ]:
!pip install sentence-transformers

In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedder = SentenceTransformerEmbeddings(
        # model_name='pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb',
        model_name = 'medicalai/ClinicalBERT'
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_chroma import Chroma
from langchain_community.vectorstores.chroma import Chroma

#Text splitter: Determinar el mejor tamaño de chunks (con este salen 14 chunks, mismo num de registros)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

23474

In [ ]:
splits

[Document(page_content='cholera due to vibrio cholerae 01, biovar cholerae', metadata={'ids': 'A00.0_cholera due to vibrio cholerae 01, biovar cholerae'}),
 Document(page_content='cholera due to vibrio cholerae 01, biovar eltor', metadata={'ids': 'A00.1_cholera due to vibrio cholerae 01, biovar eltor'}),
 Document(page_content='cholera, unspecified', metadata={'ids': 'A00.9_cholera, unspecified'}),
 Document(page_content='typhoid fever, unspecified', metadata={'ids': 'A01.00_typhoid fever, unspecified'}),
 Document(page_content='typhoid meningitis', metadata={'ids': 'A01.01_typhoid meningitis'}),
 Document(page_content='typhoid fever with heart involvement', metadata={'ids': 'A01.02_typhoid fever with heart involvement'}),
 Document(page_content='typhoid pneumonia', metadata={'ids': 'A01.03_typhoid pneumonia'}),
 Document(page_content='typhoid arthritis', metadata={'ids': 'A01.04_typhoid arthritis'}),
 Document(page_content='typhoid osteomyelitis', metadata={'ids': 'A01.05_typhoid oste

In [ ]:
print(len(splits[0].page_content))
print(splits[0].metadata)

50
{'ids': 'A00.0_cholera due to vibrio cholerae 01, biovar cholerae'}


In [ ]:
# VDB
vector_db_path = '/content/drive/My Drive/Data/vdb/'
vectorstore = Chroma.from_documents(documents=splits, embedding=embedder, persist_directory=vector_db_path)

In [ ]:
# retriever: buscar chunks relevantes basandose en una query
retriever = vectorstore.as_retriever() #ver params de búsqueda ('similarity')

### Prompt template

In [ ]:
template = '''[INST] <<SYS>>

## TASK:
- Your objective is to identify the corresponding ICD-10 codes given a medical record.
- The medical record is a sentence that contains different symptoms.
- Identify the symptoms in the sentence and return at least one code for each symptom.
- Return the collection of codes found in the sentence in list format.

## CONTEXT

- The following chunks, extracted from the vectorial base, contain relevant information about symptoms and their corresponding ICD-10 codes:
{context}

## PROCESSING INSTRUCTIONS:
- Analyze the sentence with attention to symptoms.
- Do not invent any information that is not present in the note.

## EXAMPLES OF ANSWERS:
- Medical record: 'The patient presents progressive weakness in the muscles of the extremities, especially in the arms and legs.'
- Symptoms: 'progressive weakness in the muscles of the extremities'
- Expected answer: 'R53.1, M62.8, M62.81'


## ANSWER FORMAT:
- If you identify at least one code, adhere strictly to the format (the following example has found two different codes):
"""
* "<code1>, <code2>"
"""
- Replace <code1> with the ICD-10 code of the first symptom.
- Replace <code2> with the ICD-10 code of the second symptom.
- If you do not identify any code, return: 'None'.

<</SYS>>
## NOTE:
```
{nota}
```
[/INST]'''

### Dataframe de prompts a meter en el LLM (notas)


Habría que concatenar los casos que quiero que procese - falta por hacer

In [ ]:
csv_path = '/content/drive/MyDrive/Data/DS_nuestro_statements_cortos.csv'

In [ ]:
df_free_text = pd.read_csv(csv_path, sep=',',encoding='utf-8-sig', header=0)

In [ ]:
df_free_text.head()

,Prompts,CodigosCIE
0,progressive weakness in the muscles,"R53.1,M62.81"
1,muscular debility in the limbs,"R53.1,M62.81"
2,lack of balance,"R27.8,R27.9"
3,insufficient muscle coordination,"R27.8,R27.9"
4,lack of muscle control,R27.0


In [ ]:
df = df_free_text.copy()

In [ ]:
df = df[['Prompts', 'CodigosCIE']]

In [ ]:
df.head()

,Prompts,CodigosCIE
0,progressive weakness in the muscles,"R53.1,M62.81"
1,muscular debility in the limbs,"R53.1,M62.81"
2,lack of balance,"R27.8,R27.9"
3,insufficient muscle coordination,"R27.8,R27.9"
4,lack of muscle control,R27.0


In [ ]:
df.columns = df.columns.str.strip()  # Eliminar espacios en blanco alrededor de los nombres de las columnas
df['CodigosCIE'] = df['CodigosCIE'].str.replace(' ', '')
# df['codigosSNOMED'] = df['codigosSNOMED'].str.replace(' ', '')

In [ ]:
df['Prompts'] = df['Prompts'].str.strip().str.lower()
df['codigosCIE10_pred'] = ""

In [ ]:
# df['CodigosCIE'] = df['CodigosCIE'].str.replace('.', '')

In [ ]:
df.head()

,Prompts,CodigosCIE,codigosCIE10_pred
0,progressive weakness in the muscles,"R53.1,M62.81",
1,muscular debility in the limbs,"R53.1,M62.81",
2,lack of balance,"R27.8,R27.9",
3,insufficient muscle coordination,"R27.8,R27.9",
4,lack of muscle control,R27.0,


#### DEMO

In [ ]:
# para configurar el contexto que se va a pasar al prompt (una vez obtenido el contenido relevante)
def parse_docs(docs):
  texts = ""
  for doc in docs:
      merged_cie10_id = doc.metadata.get("ids")
      cie10_id = merged_cie10_id.split("_")[1].capitalize()
      texts += f"\n{doc.page_content} ({cie10_id})"# string que contiene todas las descriptions separadas por \n
  return texts

def crear_contexto(nota:str)-> str:
  retrieved_context = retriever.get_relevant_documents(nota, top_k=5) #busca los 5 mejores chunks de la VDB antes creada
  context_text = parse_docs(retrieved_context)
  return context_text

def crear_prompt(template: str) -> PromptTemplate:
  # prompt = template.format(
  #     context=context_text,
  #     nota=nota)
  return PromptTemplate(template=template, input_variables=["nota", 'context'])

def identificar_codigoscie(nota: str, llm_chain: HuggingFacePipeline) -> str:
  respuesta = llm_chain.run(nota).strip()
  # final_response = respuesta.split('\n')[-1].strip()
  return respuesta

In [ ]:
# DEMO
nota = df['Prompts'][0]
print(nota)

progressive weakness in the muscles


In [ ]:
context_text = crear_contexto(nota)
print('5 chunks más relevantes :',context_text)

5 chunks más relevantes : 
muscle wasting and atrophy, not elsewhere classified, left upper arm (Muscle wasting and atrophy, not elsewhere classified, left upper arm)
muscle wasting and atrophy, not elsewhere classified, right upper arm (Muscle wasting and atrophy, not elsewhere classified, right upper arm)
muscle wasting and atrophy, not elsewhere classified, right thigh (Muscle wasting and atrophy, not elsewhere classified, right thigh)
muscle wasting and atrophy, not elsewhere classified, left thigh (Muscle wasting and atrophy, not elsewhere classified, left thigh)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
prompt = crear_prompt(template=template)

In [ ]:
print('template:', prompt.template)

template: [INST] <<SYS>>

## TASK:
- Your objective is to identify the corresponding ICD-10 codes given a medical record.
- The medical record is a sentence that contains different symptoms.
- Identify the symptoms in the sentence and return at least one code for each symptom.
- Return the collection of codes found in the sentence in list format.

## CONTEXT

- The following chunks, extracted from the vectorial base, contain relevant information about symptoms and their corresponding ICD-10 codes:
{context}

## PROCESSING INSTRUCTIONS:
- Analyze the sentence with attention to symptoms.
- Do not invent any information that is not present in the note.

## EXAMPLES OF ANSWERS:
- Medical record: 'The patient presents progressive weakness in the muscles of the extremities, especially in the arms and legs.'
- Symptoms: 'progressive weakness in the muscles of the extremities'
- Expected answer: 'R53.1, M62.8, M62.81'


## ANSWER FORMAT:
- If you identify at least one code, adhere strictly to 

In [ ]:
# para ver cómo queda el prompt
prompt_formatted_str: str = prompt.format(
    context=context_text,
    nota = nota)
print('Prompt final: \n', prompt_formatted_str)

Prompt final: 
 [INST] <<SYS>>

## TASK:
- Your objective is to identify the corresponding ICD-10 codes given a medical record.
- The medical record is a sentence that contains different symptoms.
- Identify the symptoms in the sentence and return at least one code for each symptom.
- Return the collection of codes found in the sentence in list format.

## CONTEXT

- The following chunks, extracted from the vectorial base, contain relevant information about symptoms and their corresponding ICD-10 codes:

muscle wasting and atrophy, not elsewhere classified, left upper arm (Muscle wasting and atrophy, not elsewhere classified, left upper arm)
muscle wasting and atrophy, not elsewhere classified, right upper arm (Muscle wasting and atrophy, not elsewhere classified, right upper arm)
muscle wasting and atrophy, not elsewhere classified, right thigh (Muscle wasting and atrophy, not elsewhere classified, right thigh)
muscle wasting and atrophy, not elsewhere classified, left thigh (Muscle w

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
 {"context": retriever, "nota": RunnablePassthrough()}
    | llm_chain
)


In [ ]:
respuesta = rag_chain.invoke(nota)

In [ ]:
respuesta['text']

'[INST] <<SYS>>\n\n## TASK:\n- Your objective is to identify the corresponding ICD-10 codes given a medical record.\n- The medical record is a sentence that contains different symptoms.\n- Identify the symptoms in the sentence and return at least one code for each symptom.\n- Return the collection of codes found in the sentence in list format.\n\n## CONTEXT\n\n- The following chunks, extracted from the vectorial base, contain relevant information about symptoms and their corresponding ICD-10 codes:\n[Document(page_content=\'muscle wasting and atrophy, not elsewhere classified, left upper arm\', metadata={\'ids\': \'M62.522_muscle wasting and atrophy, not elsewhere classified, left upper arm\'}), Document(page_content=\'muscle wasting and atrophy, not elsewhere classified, right upper arm\', metadata={\'ids\': \'M62.521_muscle wasting and atrophy, not elsewhere classified, right upper arm\'}), Document(page_content=\'muscle wasting and atrophy, not elsewhere classified, right thigh\', m

In [ ]:
start_index = respuesta['text'].find('## NOTE:') + len('## NOTE:')
final_response = respuesta['text'][start_index:].strip()
codes_start_index = respuesta['text'].find('[/INST]')
codes_final_response = respuesta['text'][codes_start_index:].strip()
codes_final_response

'[/INST] ```python\nM62.8, M62.81\n```'

In [ ]:
pattern = r'\b[A-Z]\d{1,2}(?:\.\d{1,3})?\b'
codes = re.findall(pattern, codes_final_response)
codes_unique = list(set(codes))
print(codes_unique)

['M62.8', 'M62.81']


In [ ]:
df.head()

,Prompts,CodigosCIE,codigosCIE10_pred
0,progressive weakness in the muscles,"R53.1,M62.81",
1,muscular debility in the limbs,"R53.1,M62.81",
2,lack of balance,"R27.8,R27.9",
3,insufficient muscle coordination,"R27.8,R27.9",
4,lack of muscle control,R27.0,


### Funciones auxiliares: crear prompt, crear contexto, identificar codigos, parse docs

In [ ]:
# para configurar el contexto que se va a pasar al prompt (una vez obtenido el contenido relevante)
def parse_docs(docs):
  texts = ""
  for doc in docs:
      merged_cie10_id = doc.metadata.get("ids")
      cie10_id = merged_cie10_id.split("_")[1].capitalize()
      texts += f"\n{doc.page_content} ({cie10_id})"# string que contiene todas las descriptions separadas por \n
  return texts

def crear_contexto(nota:str)-> str:
  retrieved_context = retriever.get_relevant_documents(nota, top_k=5) #busca los 5 mejores chunks de la VDB antes creada
  context_text = parse_docs(retrieved_context)
  return context_text

def crear_prompt(template: str) -> PromptTemplate:
  # prompt = template.format(
  #     context=context_text,
  #     nota=nota)
  return PromptTemplate(template=template, input_variables=["nota", 'context'])

# def identificar_codigoscie(nota: str, llm_chain: HuggingFacePipeline) -> str:
#   respuesta = llm_chain.run(nota).strip()
#   # final_response = respuesta.split('\n')[-1].strip()
#   return respuesta


In [ ]:
for idx, fila in tqdm(df.iterrows(), total=len(df), desc="Identificando codigos CIE..."):

  # crear un PromptTemplate
  prompt = crear_prompt(template=template)

  # crear una llm chain: sequence of operations that allows us to invoke a query
  llm_chain = LLMChain(prompt=prompt, llm=llm)

  # prompt específico
  nota = fila[0]

  # contexto específico (no es necesario, se busca solo con rag_chain después)
  context_text = crear_contexto(nota)

  # prompt final
  prompt_formatted_str: str = prompt.format(
    context=context_text,
    nota = nota)
  # print('\n Prompt final: \n', prompt_formatted_str)
  # print('*'*15)

  # RAG
  rag_chain = (
    {"context": retriever, "nota": RunnablePassthrough()}
        | llm_chain
    )
  respuesta = rag_chain.invoke(nota)
  print('\nPrompt y Respuesta final: ', respuesta['text'])
  print('*'*15)

  # Procesamiento de la respuesta
  codes_start_index = respuesta['text'].find('[/INST]')
  codes_final_response = respuesta['text'][codes_start_index:].strip()
  pattern = r'\b[A-Z]\d{2}(?:\.\d*[A-Z]?\d*)?\b'
  codes = re.findall(pattern, codes_final_response)
  codes_unique = list(set(codes))
  print(['Los codigos son: ',codes_unique])
  print('*'*15)
  print('\n')

  codes_str = ', '.join(codes_unique)
  df.loc[idx, "codigosCIE10_pred"] = codes_str

#   print(f'''
# > Nota:
# {nota}

# > Respuesta:
# {codes_unique}

# ----''')

Identificando codigos CIE...:   1%|          | 1/160 [00:06<17:15,  6.51s/it]


Prompt y Respuesta final:  [INST] <<SYS>>

## TASK:
- Your objective is to identify the corresponding ICD-10 codes given a medical record.
- The medical record is a sentence that contains different symptoms.
- Identify the symptoms in the sentence and return at least one code for each symptom.
- Return the collection of codes found in the sentence in list format.

## CONTEXT

- The following chunks, extracted from the vectorial base, contain relevant information about symptoms and their corresponding ICD-10 codes:
[Document(page_content='muscle wasting and atrophy, not elsewhere classified, left upper arm', metadata={'ids': 'M62.522_muscle wasting and atrophy, not elsewhere classified, left upper arm'}), Document(page_content='muscle wasting and atrophy, not elsewhere classified, right upper arm', metadata={'ids': 'M62.521_muscle wasting and atrophy, not elsewhere classified, right upper arm'}), Document(page_content='muscle wasting and atrophy, not elsewhere classified, right thigh',

Identificando codigos CIE...:   1%|▏         | 2/160 [00:11<15:03,  5.72s/it]


Prompt y Respuesta final:  [INST] <<SYS>>

## TASK:
- Your objective is to identify the corresponding ICD-10 codes given a medical record.
- The medical record is a sentence that contains different symptoms.
- Identify the symptoms in the sentence and return at least one code for each symptom.
- Return the collection of codes found in the sentence in list format.

## CONTEXT

- The following chunks, extracted from the vectorial base, contain relevant information about symptoms and their corresponding ICD-10 codes:
[Document(page_content='lesion of sciatic nerve, bilateral lower limbs', metadata={'ids': 'G57.03_lesion of sciatic nerve, bilateral lower limbs'}), Document(page_content='diplegia of upper limbs', metadata={'ids': 'G83.0_diplegia of upper limbs'}), Document(page_content='causalgia of bilateral upper limbs', metadata={'ids': 'G56.43_causalgia of bilateral upper limbs'}), Document(page_content='causalgia of bilateral lower limbs', metadata={'ids': 'G57.73_causalgia of bilater

Identificando codigos CIE...:   2%|▏         | 3/160 [00:25<25:04,  9.59s/it]

...


Prompt y Respuesta final:  [INST] <<SYS>>

## TASK:
- Your objective is to identify the corresponding ICD-10 codes given a medical record.
- The medical record is a sentence that contains different symptoms.
- Identify the symptoms in the sentence and return at least one code for each symptom.
- Return the collection of codes found in the sentence in list format.

## CONTEXT

- The following chunks, extracted from the vectorial base, contain relevant information about symptoms and their corresponding ICD-10 codes:
[Document(page_content='pain in right knee', metadata={'ids': 'M25.561_pain in right knee'}), Document(page_content='pain in left knee', metadata={'ids': 'M25.562_pain in left knee'}), Document(page_content='pain in right hip', metadata={'ids': 'M25.551_pain in right hip'}), Document(page_content='pain in left hip', metadata={'ids': 'M25.552_pain in left hip'})]

## PROCESSING INSTRUCTIONS:
- Analyze the sentence with attention to symptoms.
- Do not invent any information th

Identificando codigos CIE...:  99%|█████████▉| 159/160 [31:46<00:11, 11.08s/it]


Prompt y Respuesta final:  [INST] <<SYS>>

## TASK:
- Your objective is to identify the corresponding ICD-10 codes given a medical record.
- The medical record is a sentence that contains different symptoms.
- Identify the symptoms in the sentence and return at least one code for each symptom.
- Return the collection of codes found in the sentence in list format.

## CONTEXT

- The following chunks, extracted from the vectorial base, contain relevant information about symptoms and their corresponding ICD-10 codes:
[Document(page_content='weakness', metadata={'ids': 'R53.1_weakness'}), Document(page_content='sneezing', metadata={'ids': 'R06.7_sneezing'}), Document(page_content='geographic tongue', metadata={'ids': 'K14.1_geographic tongue'}), Document(page_content='extreme poverty', metadata={'ids': 'Z59.5_extreme poverty'})]

## PROCESSING INSTRUCTIONS:
- Analyze the sentence with attention to symptoms.
- Do not invent any information that is not present in the note.

## EXAMPLES OF A

Identificando codigos CIE...: 100%|██████████| 160/160 [32:02<00:00, 12.02s/it]


Prompt y Respuesta final:  [INST] <<SYS>>

## TASK:
- Your objective is to identify the corresponding ICD-10 codes given a medical record.
- The medical record is a sentence that contains different symptoms.
- Identify the symptoms in the sentence and return at least one code for each symptom.
- Return the collection of codes found in the sentence in list format.

## CONTEXT

- The following chunks, extracted from the vectorial base, contain relevant information about symptoms and their corresponding ICD-10 codes:
[Document(page_content='inflammatory conditions of jaws', metadata={'ids': 'M27.2_inflammatory conditions of jaws'}), Document(page_content='acute inflammatory disease of uterus', metadata={'ids': 'N71.0_acute inflammatory disease of uterus'}), Document(page_content='chronic inflammatory disease of uterus', metadata={'ids': 'N71.1_chronic inflammatory disease of uterus'}), Document(page_content='multisystem inflammatory syndrome', metadata={'ids': 'M35.81_multisystem inflamm

In [ ]:
df[0:30]

,Prompts,CodigosCIE,codigosCIE10_pred
0,progressive weakness in the muscles,"R53.1,M62.81","M62.8, M62.81"
1,muscular debility in the limbs,"R53.1,M62.81","M62.8, M62.81"
2,lack of balance,"R27.8,R27.9",R27.8
3,insufficient muscle coordination,"R27.8,R27.9",
4,lack of muscle control,R27.0,"M63.811, M62.49"
5,insufficient balance,R27.0,
6,slower speach,"R47.01,R47.1","R10.2, R15.0"
7,comunication has decelerated,"R47.01,R47.1",R13.0
8,slurred speach,R47.81,"K14.5, R47.81"
9,comunication has become indistinct,R47.81,


### Procesar resultados

In [ ]:
# def procesar_respuesta_llm(respuestas_df):
#     """Procesa una respuesta del LLM y devuelve una lista de campos."""
#     resultados = []
#     df_final = df.copy()
#     pattern = r'\b[A-Z]\d{2}(?:\.\d)?\b'
#     for idx,linea in enumerate(df['codigosCIE10_pred']):
#       codes = re.findall(pattern, linea)
#       # print(codes)
#       codes_unique = list(set(codes))
#       # print(codes_unique)
#       resultados.extend(codes_unique)
#       df_final.at[idx, 'y_pred'] = ', '.join(codes_unique)


#     return df_final, resultados

In [ ]:
# df_final, resultados= procesar_respuesta_llm(df)


In [ ]:
# df_final = df_final.drop('codigosCIE10_pred', axis=1)

In [ ]:
df['CodigosCIE'] = df['CodigosCIE'].str.replace('-', ', ')

In [ ]:
df

,Prompts,CodigosCIE,codigosCIE10_pred
0,progressive weakness in the muscles,"R53.1,M62.81","M62.8, M62.81"
1,muscular debility in the limbs,"R53.1,M62.81","M62.8, M62.81"
2,lack of balance,"R27.8,R27.9",R27.8
3,insufficient muscle coordination,"R27.8,R27.9",
4,lack of muscle control,R27.0,"M63.811, M62.49"
...,...,...,...
155,queasiness,R11.0,
156,sharp pain in the right knee,M25.561,M25.561
157,acute discomfort in the right knee,M25.561,M25.561
158,swelling,R60.9,"R06.01, J42.0"


### Descarga de resultados

In [ ]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# cwd = os.getcwd()
# csv_path = os.path.join(cwd, 'Mistral_CIE10_130424.csv')
# df_final.to_csv(csv_path, index=False)
import os
from datetime import datetime

# Obtener la fecha y hora actual
now = datetime.now()

# Formatear la fecha y hora actual según el formato deseado
date_time = now.strftime("%d%m%y_%H%M")

# FORMATO: 'LLM_CIE10_Embedder_TipoExecution_Fecha_Hora.csv'
# Definir la ruta donde deseas guardar el archivo CSV en Google Drive
typeExe = 'ClinicalBERT'
csv_path = f'/content/drive/My Drive/Data/Resultados_Mistral/Mistral_CIE10_{typeExe}_RAG_{date_time}.csv'

# Guardar el DataFrame como un archivo CSV
df.to_csv(csv_path, index=False)